# Proyecto Parte III - Gym dataset (Google Colab)

Autor: `Perez`

Objetivo: predecir la columna objetivo **`Workout type`** usando el dataset proporcionado `gym_members_exercise_tracking.csv`.

Este notebook está listo para ejecutar en **Google Colab**. Contiene:
1. Carga de datos (intenta usar el archivo incluido en el entorno).  
2. Preprocesamiento (limpieza básica, codificación).  
3. Feature selection (SelectKBest y SelectFromModel).  
4. Entrenamiento de modelo de clasificación (RandomForest).  
5. Métricas y validación (train/test + cross-validation).  
6. Conclusiones y pasos para subir a GitHub.

**Nota:** Si ejecutás en Colab y no subiste el archivo, usá el bloque de `files.upload()` que aparece abajo o montá Google Drive.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif, SelectFromModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

print('Libraries loaded')

In [ ]:
# Cargar dataset: intenta cargar desde el path proporcionado
fn = '/mnt/data/gym_members_exercise_tracking.csv'
if os.path.exists(fn):
    df = pd.read_csv(fn)
    print('Dataset cargado desde', fn)
else:
    print('No se encontró el archivo en el entorno. Ejecutá este notebook en Colab y subí el archivo usando el bloque siguiente o montá Google Drive.')
    try:
        from google.colab import files
        print('\nPor favor subí el archivo CSV ahora:')
        uploaded = files.upload()
        if len(uploaded) > 0:
            name = list(uploaded.keys())[0]
            df = pd.read_csv(name)
            print('Dataset cargado desde uploaded file:', name)
        else:
            raise FileNotFoundError('No file uploaded')
    except Exception as e:
        print('Advertencia (no Colab?):', e)
        rel = 'gym_members_exercise_tracking.csv'
        if os.path.exists(rel):
            df = pd.read_csv(rel)
            print('Dataset cargado desde', rel)
        else:
            raise FileNotFoundError('Dataset no encontrado. Coloca gym_members_exercise_tracking.csv en el mismo directorio.')

print('\nShape del dataset:', df.shape)
display(df.head())

## Exploración rápida de los datos

In [ ]:
# Información y limpieza básica
print('Columnas:')
print(df.columns.tolist())

print('\nNulos por columna:')
print(df.isna().sum())

if 'Workout type' not in df.columns:
    raise KeyError("La columna 'Workout type' no está en el dataset. Asegurate del nombre exacto (sensible a mayúsculas).")

print('\nDistribución del target (Workout type):')
print(df['Workout type'].value_counts())

print('\nTipos de datos:')
print(df.dtypes)

## Preprocesamiento
- Rellenar nulos simples
- Codificar variables categóricas
- Escalar numéricos si corresponde


In [ ]:
# Copiar df
df_proc = df.copy()

# Rellenar nulos: para columnas numéricas -> mediana; para categóricas -> 'missing'
num_cols = df_proc.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = df_proc.select_dtypes(include=['object','category']).columns.tolist()

for c in num_cols:
    df_proc[c] = df_proc[c].fillna(df_proc[c].median())
for c in cat_cols:
    df_proc[c] = df_proc[c].astype('str').fillna('missing')

# Separar X, y
y = df_proc['Workout type']
X = df_proc.drop(columns=['Workout type'])

# Codificar categóricas con LabelEncoder
from sklearn.preprocessing import LabelEncoder
encoders = {}
for c in X.select_dtypes(include=['object','category']).columns:
    le = LabelEncoder()
    X[c] = le.fit_transform(X[c].astype(str))
    encoders[c] = le

# Codificar target
le_target = LabelEncoder()
y_enc = le_target.fit_transform(y.astype(str))

print('Shape X:', X.shape)
print('Target classes:', le_target.classes_)

# Escalado opcional
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
num_in_X = X.select_dtypes(include=[np.number]).columns.tolist()
X[num_in_X] = scaler.fit_transform(X[num_in_X])

display(X.head())

## Partición train / test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_enc, test_size=0.2, stratify=y_enc, random_state=42)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)

## Feature selection 1 — SelectKBest (mutual_info_classif)

In [ ]:
k = min(15, X_train.shape[1])
skb = SelectKBest(score_func=mutual_info_classif, k=k)
skb.fit(X_train, y_train)
cols_kbest = X_train.columns[skb.get_support()].tolist()
print(f'Se seleccionaron {len(cols_kbest)} features con SelectKBest:')
print(cols_kbest)

## Feature selection 2 — SelectFromModel (RandomForest feature importances)

In [ ]:
rf_fs = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
rf_fs.fit(X_train, y_train)
selector = SelectFromModel(rf_fs, prefit=True, threshold='median')
cols_sfm = X_train.columns[selector.get_support()].tolist()
print('Features seleccionadas por SelectFromModel (threshold=median):')
print(cols_sfm)

## Entrenamiento — RandomForestClassifier
Entrenamos tres modelos: con todas las features, con SelectKBest y con SelectFromModel.

In [ ]:
def train_and_eval(clf, X_tr, X_te, y_tr, y_te):
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_te)
    scores = {
        'accuracy': accuracy_score(y_te, y_pred),
        'precision_macro': precision_score(y_te, y_pred, average='macro', zero_division=0),
        'recall_macro': recall_score(y_te, y_pred, average='macro', zero_division=0),
        'f1_macro': f1_score(y_te, y_pred, average='macro', zero_division=0)
    }
    print('Scores:', scores)
    print('\nClassification report:\n', classification_report(y_te, y_pred, zero_division=0))
    cm = confusion_matrix(y_te, y_pred)
    return scores, cm

rf = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)

print('--- All features ---')
s_all, cm_all = train_and_eval(rf, X_train, X_test, y_train, y_test)

print('\n--- SelectKBest ---')
X_train_k = skb.transform(X_train)
X_test_k = skb.transform(X_test)
s_k, cm_k = train_and_eval(rf, X_train_k, X_test_k, y_train, y_test)

print('\n--- SelectFromModel ---')
X_train_s = selector.transform(X_train)
X_test_s = selector.transform(X_test)
s_s, cm_s = train_and_eval(rf, X_train_s, X_test_s, y_train, y_test)

## Validación cruzada (StratifiedKFold, 5 folds) — comparar F1 macro

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def cv_f1(clf, X_all, y_all):
    return cross_val_score(clf, X_all, y_all, cv=cv, scoring='f1_macro')

print('CV F1 (all):', cv_f1(rf, X, y_enc).mean())
print('CV F1 (SelectKBest):', cv_f1(rf, skb.transform(X), y_enc).mean())
print('CV F1 (SelectFromModel):', cv_f1(rf, selector.transform(X), y_enc).mean())

## Matrices de confusión (test set)

In [ ]:
def plot_cm(cm, title):
    plt.figure(figsize=(5,4))
    plt.imshow(cm, interpolation='nearest')
    plt.title(title)
    plt.colorbar()
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, cm[i, j], ha='center', va='center')
    plt.show()

plot_cm(cm_all, 'Confusion matrix - All features')
plot_cm(cm_k, 'Confusion matrix - SelectKBest')
plot_cm(cm_s, 'Confusion matrix - SelectFromModel')

## Importancias de features (RandomForest usado para SelectFromModel)

In [ ]:
importances = rf_fs.feature_importances_
imp_df = pd.DataFrame({'feature': X_train.columns, 'importance': importances}).sort_values('importance', ascending=False)
imp_df_top = imp_df.head(20)
print(imp_df_top)

plt.figure(figsize=(8,4))
plt.bar(imp_df_top['feature'], imp_df_top['importance'])
plt.xticks(rotation=90)
plt.title('Top feature importances')
plt.tight_layout()
plt.show()

## Conclusiones

- Se compararon dos métodos de selección de features y un clasificador RandomForest.  
- Interpretá los resultados: fijate si con menos features (SelectKBest/SelectFromModel) mantenemos una métrica `f1_macro` cercana a la de usar todas las features.  
- Si la reducción mantiene rendimiento, preferible por interpretabilidad y menor costo computacional.

### Entrega en GitHub
1. Subí este notebook `ProyectoParteIII_Perez.ipynb` a tu repositorio.
2. Subí también el archivo `gym_members_exercise_tracking.csv`.
3. Agregá un `README.md` que explique cómo ejecutar (por ejemplo: abrir en Colab y subir el CSV o montar Drive).

---

Listo: el notebook está preparado para ejecutar en Colab. Ajustá hiperparámetros o métodos (p. ej. probar XGBoost o SVM) si lo deseás.